## Only for generation using latent vector and bidirectional lstm

In [ ]:
class EEGEncoder(nn.Module):
    def __init__(self):
        super(EEGEncoder, self).__init__()
        # LSTM layer for temporal feature extraction
        self.lstm = nn.LSTM(input_size=128, hidden_size=128, batch_first=True, bidirectional=True)

        # CNN layers for spatial feature extraction
        self.conv1 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=3, padding=1)  # BiLSTM outputs 256
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3, padding=1)

        # Fully connected layer for 128-dimensional latent vector
        self.fc = nn.Linear(64, 128)  # Final output: 128-dimensional latent space

    def forward(self, x):
        # Input shape: (batch_size, 128, 440)
        x = x.transpose(1, 2)  # Change shape to (batch_size, 440, 128) for LSTM

        # Pass through LSTM
        lstm_out, _ = self.lstm(x)  # LSTM outputs shape: (batch_size, 440, 256)

        # Transpose to (batch_size, 256, 440) for CNN
        x = lstm_out.transpose(1, 2)

        # Pass through CNN layers
        x = F.relu(self.conv1(x))  # Output: (batch_size, 128, 440)
        x = F.relu(self.conv2(x))  # Output: (batch_size, 64, 440)

        # Global average pooling to collapse temporal dimension
        x = F.adaptive_avg_pool1d(x, 1).squeeze(-1)  # Output: (batch_size, 64)

        # Fully connected layer to generate the latent vector
        latent_vector = self.fc(x)  # Output: (batch_size, 128)

        return latent_vector

In [ ]:
class Generator(nn.Module):
    def __init__(self, noise_dim=100, conditional_dim=128):
        super(Generator, self).__init__()
        self.fc = nn.Linear(noise_dim + conditional_dim, 4 * 4 * 1024)
        self.resblock1 = ResBlockUp(1024, 512)
        self.resblock2 = ResBlockUp(512, 256)
        self.resblock3 = ResBlockUp(256, 128)
        self.resblock4 = ResBlockUp(128, 64)
        self.conv = nn.Conv2d(64, 3, kernel_size=3, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, noise, condition):
        x = torch.cat([noise, condition], dim=1)  # Concatenate noise and latent vector
        x = self.fc(x).view(-1, 1024, 4, 4)
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.resblock3(x)
        x = self.resblock4(x)
        x = self.conv(x)
        return self.tanh(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, conditional_dim=128):
        super(Discriminator, self).__init__()
        self.resblock1 = ResBlockDown(3, 64)
        self.resblock2 = ResBlockDown(64, 128)
        self.resblock3 = ResBlockDown(128, 256)
        self.resblock4 = ResBlockDown(256, 512)
        self.resblock5 = ResBlockDown(512, 1024)
        self.resblock_final = ResBlockDown(1024, 1024)
        self.global_sum_pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, 1)

        # Conditional embedding
        self.embed = nn.Linear(conditional_dim, 1024)

    def forward(self, img, condition):
        x = self.resblock1(img)
        x = self.resblock2(x)
        x = self.resblock3(x)
        x = self.resblock4(x)
        x = self.resblock5(x)
        x = self.resblock_final(x)
        x = self.global_sum_pooling(x).view(x.size(0), -1)

        # Embed the condition and combine with image features
        condition_embed = self.embed(condition)
        x = x + condition_embed

        return self.fc(x)

In [ ]:
class ResBlockUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResBlockUp, self).__init__()
        # Main path
        self.block = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
        )
        # Shortcut path
        self.upsample = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1),
            nn.Upsample(scale_factor=2, mode='nearest')
        )

    def forward(self, x):
        # Main path
        main_path = F.interpolate(x, scale_factor=2, mode='nearest')
        main_path = self.block(main_path)

        # Shortcut path
        shortcut_path = self.upsample(x)

        # Combine paths
        return main_path + shortcut_path

class ResBlockDown(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResBlockDown, self).__init__()
        # Main path
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )
        # Shortcut path
        self.downsample = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2)

    def forward(self, x):
        # Main path
        main_path = self.block(x)

        # Shortcut path
        shortcut_path = self.downsample(x)

        # Combine paths
        return main_path + shortcut_path

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CUDA


In [ ]:
def train_model(encoder, generator, discriminator, data_loader, num_epochs=20, lr=0.0002, save_path="/content/drive/MyDrive/sngan-models"):
    # Ensure save directory exists
    os.makedirs(save_path, exist_ok=True)

    # Optimizers
    optimizer_encoder = torch.optim.Adam(encoder.parameters(), lr=lr)
    optimizer_gen = torch.optim.Adam(generator.parameters(), lr=lr)
    optimizer_disc = torch.optim.Adam(discriminator.parameters(), lr=lr)

    # Loss function
    gan_loss_fn = nn.BCEWithLogitsLoss()

    # Variable to track best discriminator loss
    best_disc_loss = float("inf")

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        batch_iterator = tqdm(data_loader, desc=f"Training Progress for Epoch {epoch + 1}")

        for batch in batch_iterator:
            eeg, images = batch["eeg"].to(device), batch["image"].to(device)

            # === Encode EEG to latent vector ===
            with torch.no_grad():  # Ensure no gradients are computed here
                latent_vector = encoder(eeg)  # Output: (batch_size, 128)

            # === Train Discriminator ===
            optimizer_disc.zero_grad()

            # Generate noise and fake images
            noise = torch.randn(eeg.size(0), 100).to(device)
            with torch.no_grad():  # No gradients for generator during discriminator training
                fake_images = generator(noise, latent_vector)

            # Prepare real and fake labels
            real_labels = torch.ones(eeg.size(0), 1).to(device)
            fake_labels = torch.zeros(eeg.size(0), 1).to(device)

            # Compute Discriminator loss
            real_loss = gan_loss_fn(discriminator(images, latent_vector), real_labels)
            fake_loss = gan_loss_fn(discriminator(fake_images.detach(), latent_vector), fake_labels)
            disc_loss = real_loss + fake_loss
            disc_loss.backward()
            optimizer_disc.step()

            # === Train Generator ===
            optimizer_gen.zero_grad()

            # Generate noise and fake images again for generator training
            noise = torch.randn(eeg.size(0), 100).to(device)
            fake_images = generator(noise, latent_vector)  # Recompute fake images
            gen_loss = gan_loss_fn(discriminator(fake_images, latent_vector), real_labels)
            gen_loss.backward()
            optimizer_gen.step()

            # Print losses for monitoring
            print(f"Discriminator Loss: {disc_loss.item():.4f} | Generator Loss: {gen_loss.item():.4f}")

            # Update tqdm with loss values
            batch_iterator.set_postfix({
                "Gen Loss": f"{gen_loss.item():.4f}",
                "Disc Loss": f"{disc_loss.item():.4f}"
            })

            # Save the models if discriminator loss improves
            if disc_loss.item() < best_disc_loss:
                best_disc_loss = disc_loss.item()

                # Save Generator and Discriminator models
                gen_save_path = os.path.join(save_path, "gen.pth")
                disc_save_path = os.path.join(save_path, "disc.pth")
                torch.save(generator.state_dict(), gen_save_path)
                torch.save(discriminator.state_dict(), disc_save_path)

                print(f"New best model saved with Disc Loss: {best_disc_loss:.4f}")


In [ ]:
encoder = EEGEncoder().to(device)
generator = Generator(noise_dim=100, conditional_dim=128).to(device)
discriminator = Discriminator(conditional_dim=128).to(device)

In [ ]:
# Hyperparameters
num_epochs = 20
learning_rate = 0.0002
batch_size = 32

In [ ]:
train_model(encoder, generator, discriminator, data_loader, num_epochs=20, lr=0.0002, save_path="/content/drive/MyDrive/sngan-models")

Epoch 1/20


Training Progress for Epoch 1:   0%|          | 0/748 [00:18<?, ?it/s, Gen Loss=3.8522, Disc Loss=1.3813]

Discriminator Loss: 1.3813 | Generator Loss: 3.8522


Training Progress for Epoch 1:   0%|          | 1/748 [00:19<3:57:42, 19.09s/it, Gen Loss=3.8522, Disc Loss=1.3813]

New best model saved with Disc Loss: 1.3813


...

Discriminator Loss: 0.0000 | Generator Loss: 24.4692


Training Progress for Epoch 1:  52%|█████▏    | 390/748 [1:31:24<1:15:45, 12.70s/it, Gen Loss=27.4328, Disc Loss=0.0000]

Discriminator Loss: 0.0000 | Generator Loss: 27.4328


Training Progress for Epoch 1:  52%|█████▏    | 390/748 [1:31:31<1:24:00, 14.08s/it, Gen Loss=27.4328, Disc Loss=0.0000]


KeyboardInterrupt: 

In [ ]:
# Load Generator
generator = Generator()
generator.load_state_dict(torch.load("/content/drive/MyDrive/sngan-models/gen.pth"))
generator.to(device)

# Load Discriminator
discriminator = Discriminator()
discriminator.load_state_dict(torch.load("/content/drive/MyDrive/sngan-models/disc.pth"))
discriminator.to(device)